In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from utils import load_data, plot_history_tf, plot_heat_map
from pathlib import Path
import os


In [4]:
# Config
IMG_SIZE = 128
RATIO = 0.2
RANDOM_SEED = 42
EPOCHS = 25
BATCH_SIZE = 64
NUM_CLASSES = 15
NPY_IMAGE_DIR = Path("./ecg_images_npy_128")


In [5]:
# Load 1D data
X_train, X_test, y_train, y_test = load_data(RATIO, RANDOM_SEED)

# Convert 1D -> 2D images (assumes you already generated ecg_images_npy with previous scripts)
train_image_paths = [NPY_IMAGE_DIR / f"sig_{i:06d}_lbl{y_train[i]}.npy" for i in range(len(y_train))]
test_image_paths  = [NPY_IMAGE_DIR / f"sig_{i:06d}_lbl{y_test[i]}.npy" for i in range(len(y_test))]

X_train_imgs = np.array([np.load(p).astype(np.float32)/255.0 for p in train_image_paths])
X_test_imgs  = np.array([np.load(p).astype(np.float32)/255.0 for p in test_image_paths])

loading the ecg data of No.100
loading the ecg data of No.101
loading the ecg data of No.103
loading the ecg data of No.105
loading the ecg data of No.106
loading the ecg data of No.107
loading the ecg data of No.108
loading the ecg data of No.109
loading the ecg data of No.111
loading the ecg data of No.112
loading the ecg data of No.113
loading the ecg data of No.114
loading the ecg data of No.115
loading the ecg data of No.116
loading the ecg data of No.117
loading the ecg data of No.119
loading the ecg data of No.121
loading the ecg data of No.122
loading the ecg data of No.123
loading the ecg data of No.124
loading the ecg data of No.200
loading the ecg data of No.201
loading the ecg data of No.202
loading the ecg data of No.203
loading the ecg data of No.205
loading the ecg data of No.208
loading the ecg data of No.210
loading the ecg data of No.212
loading the ecg data of No.213
loading the ecg data of No.214
loading the ecg data of No.215
loading the ecg data of No.217
loading 

In [6]:
# Ensure channel dimension (grayscale)
# RGB - 3
# Grayscale - 1
if X_train_imgs.ndim == 3:
    X_train_imgs = X_train_imgs[..., np.newaxis]
    X_test_imgs  = X_test_imgs[..., np.newaxis]

In [7]:
# 2D CNN model
def build_2d_cnn(input_shape=(IMG_SIZE, IMG_SIZE, 1), num_classes=NUM_CLASSES):
    inp = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inp)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inp, out)

In [8]:
model = build_2d_cnn()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,119 (434.06 KB)

 Trainable params: 111,119 (434.06 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
cb = [callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
history = model.fit(X_train_imgs, y_train, validation_data=(X_test_imgs, y_test),
                    epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=cb)

Epoch 1/25
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 2158s 2s/step - accuracy: 0.7622 - loss: 0.8471 - val_accuracy: 0.8032 - val_loss: 0.6692
Epoch 2/25
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 1349s 1s/step - accuracy: 0.8051 - loss: 0.6050 - val_accuracy: 0.8335 - val_loss: 0.5290
Epoch 3/25
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 23450s 19s/step - accuracy: 0.8650 - loss: 0.4340 - val_accuracy: 0.8933 - val_loss: 0.3621
Epoch 4/25
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 1027s 843ms/step - accuracy: 0.9052 - loss: 0.3277 - val_accuracy: 0.9200 - val_loss: 0.2857
Epoch 5/25
1217/1217 ━━━━━━━━━━━━━━━━━━━━ 1826s 2s/step - accuracy: 0.9243 - loss: 0.2783 - val_accuracy: 0.9220 - val_loss: 0.2898
Epoch 6/25
 164/1217 ━━━━━━━━━━━━━━━━━━━━ 25:26 1s/step - accuracy: 0.9334 - loss: 0.2434

KeyboardInterrupt: 

In [ ]:

# Plot results
plot_history_tf(history)

# Confusion matrix
y_pred = np.argmax(model.predict(X_test_imgs), axis=-1)
plot_heat_map(y_test, y_pred)